https://zerojsh00.github.io/posts/BERTopic/
https://towardsdatascience.com/topic-modeling-with-bert-779f7db187e6
https://arxiv.org/pdf/2203.05794.pdf

In [27]:
# ! which python

In [28]:
# pip install bertopic

In [29]:
# pip uninstall umap


In [30]:
# pip install hdbscan

In [31]:
# pip install umap-learn

In [32]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pickle
import re
import umap.umap_ as umap
import hdbscan

In [33]:
# 파일 불러오기
with open('../../data/bert_df.pkl', 'rb') as f:
    df = pickle.load(f)


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3487 entries, 0 to 3486
Data columns (total 28 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   q_id                3487 non-null   int64         
 1   q_posttypeid        3487 non-null   object        
 2   q_acceptedanswerid  1488 non-null   float64       
 3   q_parentid          0 non-null      object        
 4   q_creationdate      3487 non-null   datetime64[ns]
 5   q_score             3487 non-null   int64         
 6   q_viewcount         3487 non-null   int64         
 7   q_owneruserid       3487 non-null   int64         
 8   q_title             3487 non-null   object        
 9   q_tags              3487 non-null   object        
 10  q_answercount       3487 non-null   int64         
 11  q_commentcount      3487 non-null   int64         
 12  q_reputation        3487 non-null   int64         
 13  q_text              3487 non-null   object      

In [35]:
# extract the  columns for text analysis
## q_body : question body
## a_body : answer body
df_qna = df[['q_id','a_id','q_text', 'a_text']]

In [36]:
df_qna.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3487 entries, 0 to 3486
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   q_id    3487 non-null   int64 
 1   a_id    3487 non-null   int64 
 2   q_text  3487 non-null   object
 3   a_text  3487 non-null   object
dtypes: int64(2), object(2)
memory usage: 109.1+ KB


In [37]:
def cleanhtml(raw_html):
  # 1.Source code in python language is hard to understand, so replace all the <code> tag first
  cleantext_1 = re.findall(r'(?<=\<code>)(.*?)(?=<\/code>)', raw_html.replace('\n', '_**_'))
  cleantext_1 = [x.replace('_**_', '\n') for x in cleantext_1]
  # 2. replace html tags
  # <p>
  tag_re = re.compile('<.*?>')
  cleantext_2 = [re.sub(tag_re, '', x) for x in cleantext_1]
  return cleantext_2

In [38]:
# apply the function, cleanhtml to the question and body text
df_qna.loc[:, 'q_prep_text'] = df_qna['q_text'].apply(cleanhtml)
df_qna.loc[:, 'a_prep_text'] = df_qna['a_text'].apply(cleanhtml)

/var/folders/lk/t243gzpx379bzd9mvx8dcsmm0000gp/T/ipykernel_5830/1671787946.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_qna.loc[:, 'q_prep_text'] = df_qna['q_text'].apply(cleanhtml)
/var/folders/lk/t243gzpx379bzd9mvx8dcsmm0000gp/T/ipykernel_5830/1671787946.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_qna.loc[:, 'a_prep_text'] = df_qna['a_text'].apply(cleanhtml)


In [39]:
df_qna.loc[:, [ 'q_id','q_prep_text', 'a_prep_text']] 

,q_id,q_prep_text,a_prep_text
0,77593805,[],[n = 20000\nsum_of_numbers = (n * (n + 1)) // ...
1,77593717,[import hashlib\n\nuser_hash_dict = {}\n\nwith...,[from hashlib import sha256 as SHA256\n\ncpd =...
2,77591118,[London:Alpha\nLondon\nLondon:Beta\nLondon:Del...,"[s = pd.Series(['London:Alpha', 'London', 'Lon..."
3,77590853,"[-v, -o, -A, --script, -Pn, -IL]","[pip install python-nmap\n, import nmap\n\nde..."
4,77591142,[],[]
...,...,...,...
3482,77581497,[df\n\nDevice int In Out Bw_in Bw_out\n...,"[duckdb, import duckdb\n\nconn = duckdb.connec..."
3483,77580251,[55297173-0087-1 \n56397873-0186 \n57885358-...,[ df=pd.DataFrame([x.strip() for x in '''552...
3484,77567490,"[onnxruntime-silicon, onnxruntime, Flask, # Se...","[load_model, post_worker_init, sess = None\n\n..."
3485,77582066,[],"[Day, Store ID, Worker ID, # create datetime o..."


In [40]:
df_q_src = df_qna[['q_id', 'q_prep_text']].apply(pd.Series.explode)
df_a_src = df_qna[['a_id', 'a_prep_text']].apply(pd.Series.explode)

In [41]:
df_q_src = df_q_src.reset_index(drop=True)
df_a_src = df_a_src.reset_index(drop=True)

In [42]:
df_q_src['q_prep_text'] = df_q_src['q_prep_text'].str.replace('\n', ' ')
df_a_src['a_prep_text'] = df_a_src['a_prep_text'].str.replace('\n', ' ')

In [43]:
df_q_src.dropna(inplace=True)
df_a_src.dropna(inplace=True)

In [44]:
print(df_q_src['q_prep_text'].isna().sum())
print(df_a_src['a_prep_text'].isna().sum())

0
0


In [19]:
# df_q_src['q_prep_text'] = df_q_src['q_prep_text'].apply(lambda x : [xx.lstrip().rstrip() for xx in x])
# df_a_src['a_prep_text'] = df_a_src['a_prep_text'].apply(lambda x : [xx.lstrip().rstrip() for xx in x])

In [20]:
# 전체 데이터셋 
with open('../../data/q_src_df.pkl', 'wb') as f:
    pickle.dump(df_q_src, f)

with open('../../data/a_src_df.pkl', 'wb') as f:
    pickle.dump(df_a_src, f)

In [49]:
src_list = df_q_src.loc[:, 'q_prep_text'].values


In [50]:
src_list

array(["import hashlib  user_hash_dict = {}  with open('common_passwords.txt','r' ) as f:     common_passwords = f.read().splitlines()  with open('user_hash.txt', 'r') as f:     text = f.read().splitlines()     for user_hash in text:         username = user_hash.split(&quot;:&quot;)[0]         hash = user_hash.split(&quot;:&quot;)[1]         user_hash_dict[username] = hash  for password in common_passwords:     hashed_password = hashlib.sha256(password.encode('utf-8')).hexdigest()     for username , hash in user_hash_dict.items():         if hashed_password == hash:             print(f'hash found\\n{username}:{password}') ",
       'London:Alpha London London:Beta London:Delta Paris ',
       'London_sub:Alpha London_sub London_sub:Beta London_sub:Delta Paris_sub ',
       ..., '&gt;&gt;&gt; [ERROR] Worker (pid:10517) was sent SIGSEGV! ',
       "&gt;&gt;&gt; requests.exceptions.ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')

In [51]:
from transformers import AutoTokenizer, AutoModel
import torch

In [52]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
model = AutoModel.from_pretrained("microsoft/codebert-base")
nl_tokens=tokenizer.tokenize("return maximum value")

code_tokens=tokenizer.tokenize(src_list)

tokens=[tokenizer.cls_token]+nl_tokens+[tokenizer.sep_token]+code_tokens+[tokenizer.eos_token]

tokens_ids=tokenizer.convert_tokens_to_ids(tokens)

context_embeddings=model(torch.tensor(tokens_ids)[None,:])[0]


TypeError: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]

In [57]:
print(tokenizer.cls_token)
print(tokenizer.sep_token)
print(tokenizer.eos_token)

<s>
</s>
</s>


In [ ]:
context_embeddings = context_embeddings.reshape(-1,768)

In [ ]:
context_embeddings.shape

In [ ]:

umap_embeddings = umap.UMAP(n_neighbors=15,
                            n_components=5,
                            metric='cosine').fit_transform(context_embeddings.detach().numpy())




In [ ]:
cluster = hdbscan.HDBSCAN(min_cluster_size=15,
                          metric='euclidean',
                          cluster_selection_method='eom').fit(umap_embeddings)

In [ ]:
cluster

In [ ]:
.detach().numpy()